In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'D:\学习\s9s10\Apprentissage statistique\TP\TP-SVM\report':
  os.chdir(r'D:\学习\s9s10\Apprentissage statistique\TP\TP-SVM\report')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\importlib\\_bootstrap.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\importlib\\_bootstrap_external.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\zipimport.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\codecs.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\encodings\\aliases.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\encodings\\__init__.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\encodings\\utf_8.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\encodings\\gbk.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\abc.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\io.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\stat.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\_collections_abc.py": 1748965307.0, "C:\\Users\\18219\\.conda\\envs\\TP_SVM\\Lib\\gen

In [2]:
#| echo: false
# --- Imports et configuration ---
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from time import time
scaler = StandardScaler()

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

import sys, os
sys.path.insert(0, os.path.abspath('../src'))

from svm_source import *

In [3]:
#| fig-pos: H
#| out-width: 55%

###################################################
#               Iris Dataset
###################################################
iris = datasets.load_iris()
X = iris.data
X = scaler.fit_transform(X)
y = iris.target
X = X[y != 0, :2]
y = y[y != 0]

# Visualization
fig, ax = plt.subplots()
plot_2d(X, y)              
plt.tight_layout()
plt.show()

# split train test (say 25% for the test)
# using train_test_split whithout shuffling (fix the random state = 42) for reproductibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

<Figure size 1650x1050 with 1 Axes>

In [4]:
# Q1 Linear kernel

# fit the model and select the best hyperparameter C
parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 200))}
clf1 = SVC()
clf_linear = GridSearchCV(clf1, parameters, n_jobs=-1)
clf_linear.fit(X_train, y_train)

# compute the score
print('Generalization score for linear kernel: %s, %s' %
      (clf_linear.score(X_train, y_train),
       clf_linear.score(X_test, y_test)))

Generalization score for linear kernel: 0.7466666666666667, 0.68


In [5]:
# Q2 polynomial kernel
Cs = list(np.logspace(-3, 3, 5))
gammas = 10. ** np.arange(1, 2)
degrees = np.r_[1, 2, 3]

# fit the model and select the best set of hyperparameters
parameters = {'kernel': ['poly'], 'C': Cs, 'gamma': gammas, 'degree': degrees}
clf2 = SVC()
clf_poly = GridSearchCV(clf2, parameters, n_jobs=-1)
clf_poly.fit(X_train, y_train)

print(clf_poly.best_params_)
print('Generalization score for polynomial kernel: %s, %s' %
      (clf_poly.score(X_train, y_train),
       clf_poly.score(X_test, y_test)))

{'C': np.float64(0.03162277660168379), 'degree': np.int64(1), 'gamma': np.float64(10.0), 'kernel': 'poly'}
Generalization score for polynomial kernel: 0.7466666666666667, 0.68


In [6]:
# display the results using frontiere
def f_linear(xx):
    """Classifier: needed to avoid warning due to shape issues"""
    return clf_linear.predict(xx.reshape(1, -1))

def f_poly(xx):
    return clf_poly.predict(xx.reshape(1, -1))

# display the frontiere
plt.ion()
plt.figure(figsize=(15, 5))
plt.subplot(131)
plot_2d(X, y)
plt.title("iris dataset")

plt.subplot(132)
frontiere(f_linear, X, y)
plt.title("linear kernel")

plt.subplot(133)
frontiere(f_poly, X, y)

plt.title("polynomial kernel")
plt.tight_layout()
plt.draw()

<Figure size 4500x1500 with 5 Axes>

In [7]:
#| echo: false

###################################################
#               Face Recognition Task
###################################################


####################################################################
# Download the data and unzip; then load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4,
                              color=True, funneled=False, slice_=None,
                              download_if_missing=True)
# data_home='.'

# introspect the images arrays to find the shapes (for plotting)
images = lfw_people.images
n_samples, h, w, n_colors = images.shape

# the label to predict is the id of the person
target_names = lfw_people.target_names.tolist()

####################################################################
# Pick a pair to classify such as
names = ['Tony Blair', 'Colin Powell']
# names = ['Donald Rumsfeld', 'Colin Powell']

idx0 = (lfw_people.target == target_names.index(names[0]))
idx1 = (lfw_people.target == target_names.index(names[1]))
images = np.r_[images[idx0], images[idx1]]
n_samples = images.shape[0]
y = np.r_[np.zeros(np.sum(idx0)), np.ones(np.sum(idx1))].astype(int)

# plot a sample set of the data
plot_gallery(images, np.arange(12))
plt.show()

<Figure size 2160x2160 with 12 Axes>

In [8]:
####################################################################
# Extract features

# features using only illuminations
X = (np.mean(images, axis=3)).reshape(n_samples, -1)

# # or compute features using colors (3 times more features)
# X = images.copy().reshape(n_samples, -1)

# Scale features
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

In [9]:
####################################################################
# Split data into a half training and half test set
# X_train, X_test, y_train, y_test, images_train, images_test = \
#    train_test_split(X, y, images, test_size=0.5, random_state=0)
# X_train, X_test, y_train, y_test = \
#    train_test_split(X, y, test_size=0.5, random_state=0)

indices = np.random.permutation(X.shape[0])
train_idx, test_idx = indices[:X.shape[0] // 2], indices[X.shape[0] // 2:]
X_train, X_test = X[train_idx, :], X[test_idx, :]
y_train, y_test = y[train_idx], y[test_idx]
images_train, images_test = images[
    train_idx, :, :, :], images[test_idx, :, :, :]

####################################################################
# Quantitative evaluation of the model quality on the test set

In [10]:
#| warning: false

# Q4
print("--- Linear kernel ---")
print("Fitting the classifier to the training set")
t0 = time()

# fit a classifier (linear) and test all the Cs
Cs = 10. ** np.arange(-5, 6)
scores = []
for C in Cs:
    clf_tmp = SVC(kernel='linear', C=C)
    clf_tmp.fit(X_train, y_train)
    scores.append(clf_tmp.score(X_test, y_test))

ind = int(np.argmax(scores))
best_C = Cs[ind]
best_acc = float(scores[ind])
best_err = 1.0 - best_acc

print("Best C: {}".format(best_C))

plt.figure()
plt.plot(Cs, scores, label="Accuracy")
plt.scatter([best_C], [best_acc], s=80, zorder=3)
plt.axvline(best_C, linestyle="--", alpha=0.6)
plt.annotate(
    "Best C={:.1e}\nacc={:.3f}".format(best_C, best_acc),
    xy=(best_C, best_acc),
    xytext=(1.5*best_C, min(1.0, best_acc + 0.05)),  
    arrowprops=dict(arrowstyle="->", lw=1),
    fontsize=10
)
plt.xlabel("Paramètres de régularisation C")
plt.ylabel("Scores d'apprentissage (accuracy)")
plt.xscale("log")
plt.legend()
plt.tight_layout()
plt.show()

print("Best score (accuracy): {}".format(best_acc))

# Erreur de prédiction
errors = 1.0 - np.array(scores)

plt.figure()
plt.plot(Cs, errors, label="Erreur de prédiction")
plt.scatter([best_C], [best_err], s=80, zorder=3)
plt.axvline(best_C, linestyle="--", alpha=0.6)
plt.annotate(
    "Best C={:.1e}\nerreur={:.3f}".format(best_C, best_err),
    xy=(best_C, best_err),
    xytext=(1.5*best_C, min(1.0, best_err + 0.05)),
    arrowprops=dict(arrowstyle="->", lw=1),
    fontsize=10
)
plt.xlabel("Paramètre de régularisation C")
plt.ylabel("Erreur de prédiction (1 - accuracy)")
plt.xscale("log")
plt.legend()
plt.tight_layout()
plt.show()


print("Predicting the people names on the testing set")
t0 = time()

--- Linear kernel ---
Fitting the classifier to the training set


Best C: 0.001


<Figure size 1650x1050 with 1 Axes>

Best score (accuracy): 0.9157894736842105


<Figure size 1650x1050 with 1 Axes>

Predicting the people names on the testing set


In [11]:
#| warning: false
# predict labels for the X_test images with the best classifier
clf = SVC(kernel='linear', C=Cs[ind])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("done in %0.3fs" % (time() - t0))
# The chance level is the accuracy that will be reached when constantly predicting the majority class.
print("Chance level : %s" % max(np.mean(y), 1. - np.mean(y)))
print("Accuracy : %s" % clf.score(X_test, y_test))

done in 0.648s
Chance level : 0.6210526315789474


Accuracy : 0.9157894736842105


In [12]:
#| warning: false
####################################################################
# Qualitative evaluation of the predictions using matplotlib

prediction_titles = [title(y_pred[i], y_test[i], names)
                     for i in range(y_pred.shape[0])]

plot_gallery(images_test, prediction_titles)
plt.show()

####################################################################
# Look at the coefficients
plt.figure()
plt.imshow(np.reshape(clf.coef_, (h, w)))
plt.show()

<Figure size 2160x2160 with 12 Axes>

<Figure size 1650x1050 with 1 Axes>

In [13]:
#| warning: false

# Q5
from sklearn import svm
def run_svm_cv(_X, _y):
    _indices = np.random.permutation(_X.shape[0])
    _train_idx, _test_idx = _indices[:_X.shape[0] // 2], _indices[_X.shape[0] // 2:]
    _X_train, _X_test = _X[_train_idx, :], _X[_test_idx, :]
    _y_train, _y_test = _y[_train_idx], _y[_test_idx]

    _parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 5))}
    _svr = svm.SVC()
    _clf_linear = GridSearchCV(_svr, _parameters)
    _clf_linear.fit(_X_train, _y_train)

    print('Generalization score for linear kernel: %s, %s \n' %
          (_clf_linear.score(_X_train, _y_train), _clf_linear.score(_X_test, _y_test)))

print("Score sans variable de nuisance")
# use run_svm_cv on original data
run_svm_cv(X, y)

print("Score avec variable de nuisance")
n_features = X.shape[1]
# On rajoute des variables de nuisances
sigma = 1
noise = sigma * np.random.randn(n_samples, 300, ) 
#with gaussian coefficients of std sigma
X_noisy = np.concatenate((X, noise), axis=1)
X_noisy = X_noisy[np.random.permutation(X.shape[0])]
# use run_svm_cv on noisy data
run_svm_cv(X_noisy, y)

Score sans variable de nuisance


Generalization score for linear kernel: 1.0, 0.8736842105263158 

Score avec variable de nuisance


Generalization score for linear kernel: 1.0, 0.5789473684210527 



In [14]:
#| warning: false

# Q6
print("Score apres reduction de dimension")

# 复用 Q4 找到的最佳 C（避免在 Q6 再次做网格搜索）
C_fixed = Cs[ind]

# 全量数据与索引
Xn_all = X_noisy
yn_all = y

# n_components 从 10 到 150，步长 10；并根据数据上界自动截断
max_nc = min(Xn_all.shape[0], Xn_all.shape[1])
grid_n_components = list(range(10, 151, 10))
grid_n_components = [k for k in grid_n_components if k <= max_nc]
if not grid_n_components:
    grid_n_components = [min(20, max_nc)]

# 只拟合一次 PCA，用最大的 k，这样循环里直接切片前 k 个主成分即可
Kmax = max(grid_n_components)
pca = PCA(n_components=Kmax, svd_solver='randomized', iterated_power=1)

# **在全量数据上拟合 PCA（会有数据泄露，但这是你希望的设置）**
Z_all = pca.fit_transform(Xn_all)   # 形状: (n_samples, Kmax)

test_scores = []
train_scores = []
best_records = []  # (k, C_fixed, train_score, test_score)

for k in grid_n_components:
    # 取前 k 个主成分，并按原索引还原 train/test
    Ztr = Z_all[train_idx, :k]
    Zte = Z_all[test_idx, :k]
    ytr = yn_all[train_idx]
    yte = yn_all[test_idx]

    # 更快的线性 SVM：LinearSVC（复用 Q4 的最佳 C）
    clf = LinearSVC(C=C_fixed, dual="auto", max_iter=5000)
    clf.fit(Ztr, ytr)

    tr = clf.score(Ztr, ytr)
    te = clf.score(Zte, yte)

    train_scores.append(tr)
    test_scores.append(te)
    best_records.append((k, C_fixed, tr, te))

# 打印最佳的 n_components 及对应分数
best_idx = int(np.argmax(test_scores))
best_k, best_C, best_tr, best_te = best_records[best_idx]
print(f"Best n_components = {best_k}, best C = {best_C}")
print(f"Train score = {best_tr:.3f}, Test score = {best_te:.3f}")

# 可视化：测试分数 vs n_components
plt.figure()
plt.plot(grid_n_components, test_scores, marker='o')
plt.xlabel("n_components (PCA, fit sur full data)")
plt.ylabel("Test accuracy (LinearSVC)")
plt.title("Impact de la dimension apres PCA (fit full) sur donnees bruitees")
plt.tight_layout()
plt.show()

Score apres reduction de dimension
Best n_components = 10, best C = 0.001
Train score = 0.589, Test score = 0.558


<Figure size 1650x1050 with 1 Axes>